**INSTALAMOS LIBRERIAS PARA CARGA**

In [ ]:
! pip install psycopg2
! pip install pgdb 

     |████████████████████████████████| 3.0 MB 7.4 MB/s 
  Created wheel for pgdb: filename=pgdb-0.0.11-py3-none-any.whl size=2800 sha256=41f58b7c3f4da6b4d06ea0c657016ab952a6230dfcb3ce0bbe0ce53d85b63804
  Stored in directory: /root/.cache/pip/wheels/ff/83/6a/8cd1726f08b2d24a023f99cdb271b156b9d61ab3771ed80e30
Successfully built pgdb


**ASIGNAMOS EL PROYECTO DONDE SE ENCUENTRA NUESTRA BD Y EL BUCKET DE LOS DATOS QUE SE CARGARAN**

In [ ]:
PROJECT_ID = "keepcoding-proyectofinal-01" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
BUCKET_NAME = "kcp-bucket-01" #@param {type:"string"}
REGION = "europe-west1" #@param {type:"string"}

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://kcp-bucket-01/datos_detection/
                                 gs://kcp-bucket-01/datos_equivalencies/
                                 gs://kcp-bucket-01/datos_images_out/
                                 gs://kcp-bucket-01/datos_processed/
                                 gs://kcp-bucket-01/datos_shopping_list/
                                 gs://kcp-bucket-01/datos_store/


**COPIAMOS LA CARPETA DATOS DE NUESTRO BUCKET A NUESTRO LOCAL PARA TRATARLOS ANTES DE REALIZAR LA CARGA**

In [ ]:
! gsutil -m cp -r gs://$BUCKET_NAME/datos_detection /content/

Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220323.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220327.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220326.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220331.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220322.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220328.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220324.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220325.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220329.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220330.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220401.csv...

LEEMOS EL FICHERO CSV DE DETECTION A UN DATAFRAME

In [ ]:
import os
import pandas as pd
import glob

files_joined = os.path.join('/content/datos_detection', '*.csv')
list_files_store = glob.glob(files_joined)
df_data_detection = pd.concat(map(pd.read_csv, list_files_store), ignore_index=True)

COMPROBAMOS LOS DATOS DEL DATAFRAME

In [ ]:
print(df_data_detection.columns)
df_data_detection.head()

Index(['id_detection', 'product', 'qty_detected', 'image_url', 'date_created'], dtype='object')


,id_detection,product,qty_detected,image_url,date_created
0,42,potato,0,https://storage.googleapis.com/kcp-bucket-01/d...,20220402
1,42,orange,2,https://storage.googleapis.com/kcp-bucket-01/d...,20220402
2,42,pineapple,0,https://storage.googleapis.com/kcp-bucket-01/d...,20220402
3,42,apple,11,https://storage.googleapis.com/kcp-bucket-01/d...,20220402
4,42,lemon,0,https://storage.googleapis.com/kcp-bucket-01/d...,20220402


GENERAMOS UN NUEVO CSV CON LOS DATOS CON EL NUEVO FORMATO. APROVECHAMOS PARA QUITAR EL INDICE.

In [ ]:
df_data_detection.to_csv('/content/datos_detection/Datos_Detection_format.csv', index=False, sep=',')

**IMPORTAMOS LIBRERIAS PARA TRABAJAR CON LA CARGA DE DATOS**

In [ ]:
import psycopg2
import pgdb

**CARGA DE DATOS EN LA TABLA SHOPLIST**

In [ ]:
def cargar(path):
    """ Conexión al servidor de pases de datos PostgreSQL """
    conexion = None
    try:
        # Conexion al servidor de PostgreSQL
        print('Conectando a la base de datos PostgreSQL...')
        conexion = pgdb.Connection(host="34.79.94.25",database="proyecto", user="postgres", password="keepcoding")
 
        # creación del cursor
        cur = conexion.cursor()

        # def_table = '''
        # DROP TABLE IF EXISTS Detection;  
        # CREATE TABLE IF NOT EXISTS Detection
        # (     id_detection TEXT NOT NULL,
        #       product TEXT NOT NULL,
        #       qty_detected NUMERIC NOT NULL,
        #       image_url TEXT NOT NULL,
        #       date_created DATE NOT NULL
        # );'''
        # cur.execute(def_table)
        # conexion.commit()

        csv_file_name = path
        sql = "COPY Detection FROM STDIN DELIMITER ',' CSV HEADER"
        cur.copy_expert(sql, open(csv_file_name, "r"))
        conexion.commit()

        # Cierre de la comunicación con PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conexion is not None:
            conexion.close()
            print('Conexión finalizada.')

In [ ]:
cargar('/content/datos_detection/Datos_Detection_format.csv')

Conectando a la base de datos PostgreSQL...
Conexión finalizada.


**SE MUEVEN LOS DATOS A LA CARPETA PROCESADOS**

In [ ]:
! gsutil mv -r gs://$BUCKET_NAME/datos_detection gs://$BUCKET_NAME/datos_processed/

Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220322.csv [Content-Type=text/csv]...
Removing gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220322.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220323.csv [Content-Type=text/csv]...
Removing gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220323.csv...
Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220324.csv [Content-Type=text/csv]...
Removing gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220324.csv...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://kcp-bucket-01/datos_detection/datos_detection_Datos_Detection_20220325.csv [Content-Type=text/csv]...
Removing gs